In [1]:
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

from collections import defaultdict
from gensim import corpora, models
from nltk.corpus import stopwords
from pprint import pprint


/Users/scott/projects/acrl_analysis/acrl/acrl_env/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/Users/scott/projects/acrl_analysis/acrl/acrl_env/lib/python3.9/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
data = pd.read_csv("data_cleaned/combined.csv")
data.head()

,id,title,format,affiliation 1,affiliation 2,affiliation 3,affiliation 4,affiliation 5,affiliation 6,affiliation 7,...,Abstract,lower_abstract,affiliation,mode of presentation,affiliation 8,affiliation 9,affiliation 10,tags (primary),tags (secondary),affiliation 11
0,2013-1,Increasing the Effectiveness of Your Scholarly...,preconference workshop,University of Utah,Tecker International,NaN,NaN,NaN,NaN,NaN,...,This workshop is designed for those that are b...,this workshop is designed for those that are b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-2,"Planning, Assessing, and Communicating Library...",preconference workshop,University of Illinois at Urbana-Champaign,Pierce College,NaN,NaN,NaN,NaN,NaN,...,Libraries in higher education are increasingly...,libraries in higher education are increasingly...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-3,"Plugged into User Behavior: Low-Budget, High-I...",preconference workshop,University of Texas Arlington,NaN,NaN,NaN,NaN,NaN,NaN,...,How do students really use library subject gui...,how do students really use library subject gui...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-4,Rediscover the Joy of Working Together!: Posit...,preconference workshop,Kansas State University,Rockhurst University,University of Kansas,University of Iowa,NaN,NaN,NaN,...,Join this collaborative session to enhance you...,join this collaborative session to enhance you...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-5,Speculation to Litigation—Copyright and Climat...,preconference workshop,University of Louisville,Columbia University,Brigham Young University,Purdue University,NaN,NaN,NaN,...,Changes in digital technologies have produced ...,changes in digital technologies have produced ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Define function to remove rows with no abstract in a dataframe
def remove_no_abstract(data):
    data = data[data["lower_abstract"] != "no abstract"]
    data = data[data["lower_abstract"] != "No abstract"]
    data = data[data["lower_abstract"] != "no abstract available"]
    return data

In [4]:
# remove rows with no abstract in a dataframe
data = remove_no_abstract(data)

In [5]:
set(data["year"].tolist())

{2007, 2009, 2011, 2013, 2015, 2017, 2019, 2021}

In [6]:
documents = data["lower_abstract"].dropna().tolist()

In [7]:
stoplist = set(stopwords.words("english"))
stoplist.update(["library", "libraries"])
print(stoplist)

{'she', 'you', 'being', 's', 'doing', 'further', "shouldn't", "couldn't", "won't", 'isn', 'or', 'been', 'here', 'himself', 'theirs', 'through', 'if', "doesn't", 'him', 'library', 'so', "isn't", "weren't", "wouldn't", 'below', 'hers', 'to', 'this', 'while', 'own', 'of', 'but', 'was', 'y', 'as', 'aren', "hasn't", 't', 'at', "she's", "hadn't", 'shouldn', 'an', 'after', 'up', 'few', 'ours', 'haven', 'be', 'a', 'just', 'down', 'each', "shan't", 'should', 'from', 'its', 'your', 'their', 'had', 'are', 'during', 'having', 'above', 'more', 'will', 'there', 'his', 'how', 'were', 'yourself', 'by', 'other', "didn't", 'ourselves', 'who', 'into', 'they', 'when', 'than', 'we', "it's", "you'd", 'for', 'our', 'any', 'll', 'hasn', 'has', 'against', 'some', 'such', 'her', 'whom', 'not', 'the', 'won', 'under', 'needn', "you've", 'me', 'did', 'that', 'myself', "you'll", 'out', "haven't", 'mustn', 'is', 'very', 'themselves', 'shan', 've', 'didn', "mightn't", 'between', 'why', 'nor', 'until', 'herself', 'whe

In [8]:
# Read custom stopwords from file and add to stoplist
with open("stopwords.txt", "r") as f:
    custom_stopwords = f.read().splitlines()
    lower_custom_stopwords = [w.lower() for w in custom_stopwords]
stoplist.update(lower_custom_stopwords)

In [9]:
stoplist

{'a',
 'about',
 'above',
 'abstract',
 'academic',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'attendees',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'campus',
 'can',
 'college',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'discuss',
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'information',
 'innovation',
 'innovative',
 'institution',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'learn',
 'learning',
 'librarian',
 'librarians',
 'libraries',
 'library',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',


In [10]:
texts = [[word for word in document.lower().split() if word not in stoplist]
            for document in documents]

In [11]:
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

In [12]:
texts = [[token for token in text if frequency[token] > 1]
            for text in texts]

In [13]:
pprint(texts[:1])

[['workshop',
  'designed',
  'building',
  'programs',
  'institutions',
  'need',
  'strengthen',
  'revitalize',
  'local',
  'efforts.',
  'emphasis',
  'placed',
  'think',
  'strategically',
  'local',
  'programs',
  'develop',
  'skills',
  'influential',
  'without',
  'necessarily',
  'positional',
  'authority.',
  'workshop',
  'encourage',
  'participant',
  'interaction',
  'incorporating',
  'hands-on',
  'activities.']]


In [14]:
dictionary = corpora.Dictionary(texts)
dictionary.save('acrl.dict')

In [15]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('acrl.mm', corpus)

In [16]:
hdp = models.HdpModel(corpus, id2word=dictionary)

In [17]:
hdp.print_topics()

[(0,
  '0.004*literacy + 0.003*data + 0.003*faculty + 0.003*digital + 0.002*use + 0.002*services + 0.002*work + 0.002*instruction + 0.002*social + 0.002*provide'),
 (1,
  '0.002*program + 0.002*assessment + 0.002*data + 0.002*literacy + 0.002*provide + 0.001*reference + 0.001*also + 0.001*panelists + 0.001*support + 0.001*digital'),
 (2,
  '0.003*assessment + 0.003*open + 0.002*liaison + 0.002*faculty + 0.002*support + 0.002*workshop + 0.002*intersections + 0.002*data + 0.002*program + 0.002*participants'),
 (3,
  '0.002*faculty + 0.002*better + 0.002*technology + 0.002*assessment + 0.002*design + 0.002*literacy + 0.001*program + 0.001*digital + 0.001*concepts + 0.001*using'),
 (4,
  '0.002*use + 0.002*provide + 0.002*explore + 0.002*services + 0.001*data + 0.001*instruction + 0.001*assessment + 0.001*open + 0.001*help + 0.001*resources'),
 (5,
  '0.002*faculty + 0.002*literacy + 0.002*digital + 0.002*support + 0.001*consortium + 0.001*approach + 0.001*develop + 0.001*study + 0.001*eng

In [18]:
pyLDAvis.enable_notebook()

In [19]:
gensimvis.prepare(hdp, corpus, dictionary)

/Users/scott/projects/acrl_analysis/acrl/acrl_env/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/scott/projects/acrl_analysis/acrl/acrl_env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/scott/projects/acrl_analysis/acrl/acrl_env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/scott/projects/acrl_analysis/acrl/acrl_env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentat

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.122483 -0.004178       1        1  32.421851
1      0.080811  0.006050       2        1   8.026132
3      0.071348  0.000554       3        1   4.322358
2      0.071773  0.001989       4        1   4.269789
4      0.067719  0.015566       5        1   4.198239
...         ...       ...     ...      ...        ...
105   -0.015831  0.014805     146        1   0.003024
146   -0.018317 -0.016814     147        1   0.002680
77    -0.024983  0.019304     148        1   0.001785
138   -0.021034  0.017331     149        1   0.001754
148   -0.026574  0.016980     150        1   0.001120

[150 rows x 5 columns], topic_info=              Term        Freq       Total  Category  logprob  loglift
168       literacy  165.000000  165.000000   Default  30.0000  30.0000
187           data  154.000000  154.000000   Default  29.0000  29.0000
165        faculty  138.000000  138.000000   Default  28.0000  28.0000
134        digital  126.000000  126.000000   Default  27.0000  27.0000
424       services  110.000000  110.000000   Default  26.0000  26.0000
...            ...         ...         ...       ...      ...      ...
1891     software.    0.000713   11.454364  Topic150  -7.2519   1.7151
5262   priorities.    0.000811   28.948069  Topic150  -7.1236   0.9162
7532       agents,    0.000692    9.995547  Topic150  -7.2826   1.8207
4264  competencies    0.000707   12.875351  Topic150  -7.2609   1.5892
6962          dead    0.000692   12.130671  Topic150  -7.2821   1.6275

[7595 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
471       1  0.404406           &
471       2  0.050551           &
471       6  0.050551           &
471       7  0.151652           &
471      10  0.050551           &
...     ...       ...         ...
7081      4  0.114397  ﬂexibility
7081      7  0.228794  ﬂexibility
7047      1  0.136557     ﬂexible
7047      2  0.136557     ﬂexible
7047      5  0.136557     ﬂexible

[15460 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 4, 3, 5, 6, 7, 8, 9, 12, 10, 14, 11, 13, 19, 16, 22, 32, 23, 24, 15, 20, 25, 28, 36, 26, 33, 21, 17, 18, 35, 27, 30, 31, 46, 38, 37, 48, 40, 41, 43, 42, 64, 34, 29, 60, 49, 51, 50, 54, 70, 39, 52, 47, 45, 57, 53, 73, 44, 88, 69, 93, 101, 55, 86, 129, 91, 84, 66, 111, 76, 127, 65, 112, 82, 71, 61, 83, 125, 63, 62, 67, 59, 104, 79, 58, 142, 123, 109, 74, 143, 107, 90, 102, 150, 87, 81, 97, 120, 114, 140, 134, 105, 108, 80, 103, 77, 94, 116, 148, 75, 89, 121, 118, 119, 117, 100, 72, 96, 126, 146, 133, 124, 135, 56, 138, 95, 128, 110, 132, 85, 92, 98, 137, 122, 144, 145, 68, 136, 99, 141, 131, 113, 115, 130, 106, 147, 78, 139, 149])

In [20]:
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=30)

In [21]:
lda.print_topics()

[(3,
  '0.010*"online" + 0.008*"community" + 0.008*"arab" + 0.007*"public" + 0.006*"budgets" + 0.005*"r." + 0.005*"author" + 0.005*"calls" + 0.005*"justify" + 0.005*"provide"'),
 (23,
  '0.010*"game" + 0.009*"workshop" + 0.007*"remix" + 0.006*"increasingly" + 0.006*"different" + 0.006*"addressing" + 0.005*"skills" + 0.005*"concrete" + 0.005*"take" + 0.005*"panelists"'),
 (6,
  '0.011*"spaces" + 0.009*"collaborative" + 0.008*"culturally" + 0.008*"humanities" + 0.008*"scholarship" + 0.007*"diversity," + 0.007*"faculty" + 0.006*"maker" + 0.005*"data" + 0.005*"support"'),
 (29,
  '0.009*"literacy" + 0.008*"faculty" + 0.007*"study" + 0.007*"teaching" + 0.006*"create" + 0.006*"discover" + 0.006*"correlation" + 0.005*"use" + 0.005*"identify" + 0.005*"counter"'),
 (13,
  '0.010*"beginners" + 0.008*"using" + 0.007*"passing" + 0.006*"service" + 0.006*"reference" + 0.006*"social" + 0.005*"many" + 0.005*"cognitive" + 0.005*"well" + 0.005*"topics,"'),
 (8,
  '0.014*"literacy" + 0.007*"large" + 0.00

In [22]:
gensimvis.prepare(lda, corpus, dictionary)

/Users/scott/projects/acrl_analysis/acrl/acrl_env/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
18     0.036471  0.063885       1        1  6.707630
4      0.083114  0.051121       2        1  6.529956
0      0.072690  0.031568       3        1  5.886104
15     0.071898  0.036212       4        1  5.819000
27     0.063084  0.024329       5        1  4.814503
11     0.038329 -0.012679       6        1  4.227311
16     0.035980 -0.046346       7        1  4.154843
1      0.034426 -0.003741       8        1  4.058693
20     0.024358 -0.009490       9        1  4.035794
8     -0.006488  0.013215      10        1  3.652657
26     0.028385 -0.072497      11        1  3.561885
29     0.016428  0.007477      12        1  3.338011
9      0.008923 -0.026229      13        1  3.308798
7      0.035137 -0.115805      14        1  3.250132
21     0.031667  0.074736      15        1  3.186545
14    -0.014892 -0.077257      16        1  2.954509
2      0.029125  0.066426      17        1  2.918317
5     -0.054039  0.049578      18        1  2.653988
13    -0.010606 -0.086727      19        1  2.531507
10    -0.057200 -0.033574      20        1  2.434790
6      0.028866 -0.049178      21        1  2.397528
24    -0.038762 -0.051155      22        1  2.381711
28    -0.019645  0.032521      23        1  2.143053
25    -0.024028  0.008050      24        1  2.088453
19    -0.031835  0.039969      25        1  2.004277
12    -0.095723  0.037807      26        1  1.897738
23    -0.021665  0.010739      27        1  1.882236
3     -0.065610 -0.050541      28        1  1.864346
22    -0.049411  0.053609      29        1  1.814703
17    -0.148976  0.033976      30        1  1.500983, topic_info=             Term        Freq       Total Category  logprob  loglift
5895  open-access   46.000000   46.000000  Default  30.0000  30.0000
598   collections  152.000000  152.000000  Default  29.0000  29.0000
728          open  250.000000  250.000000  Default  28.0000  28.0000
187          data  412.000000  412.000000  Default  27.0000  27.0000
1840      special   69.000000   69.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
126    challenges    5.409906  173.334493  Topic30  -5.5181   0.7321
1076     cultural    5.059420   60.807469  Topic30  -5.5851   1.7126
1151         time    5.116099  111.944570  Topic30  -5.5739   1.1134
296      session,    4.973364   73.542038  Topic30  -5.6022   1.5053
205       provide    4.916760  291.006542  Topic30  -5.6137   0.1184

[2040 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
471       1  0.030103            &
471       2  0.090308            &
471       5  0.361232            &
471       7  0.030103            &
471       9  0.030103            &
...     ...       ...          ...
6387     20  0.816626      “global
4510      2  0.760899         “how
3524     18  0.918237       “maker
5100     25  0.723786  “publishing
5941     26  0.749941       “state

[9141 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 5, 1, 16, 28, 12, 17, 2, 21, 9, 27, 30, 10, 8, 22, 15, 3, 6, 14, 11, 7, 25, 29, 26, 20, 13, 24, 4, 23, 18])